In [1]:
from skilift import GTFS, TransitGraph

In [2]:
import pandas as pd
from zipfile import ZipFile
from collections import defaultdict
from typing import Dict, Set
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
fn = "/home/bmander/skilift_data/transit/redding.zip"

graph = TransitGraph.load(fn)

In [4]:
graph.feed.day_start, graph.feed.day_end

(0, 70200)

In [5]:
# get epoch time on Aptil 19, 2023 at 1:00pm pacific time
t0 = pd.Timestamp("2023-04-19 13:00:00", tz="America/Los_Angeles")
t0

Timestamp('2023-04-19 13:00:00-0700', tz='America/Los_Angeles')

In [6]:
stop_id = graph.feed.stops_with_name("Downtown Transit Center").iloc[0].stop_id
stop_id

2000

In [7]:
pattern_ids = graph.feed.stop_pattern_ids[stop_id]
pattern_ids

{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 23, 24, 25, 26, 27, 28}

In [8]:
service_ids = graph.feed.get_service_ids(t0.date())
service_ids

{'c_1658_b_18260_d_31',
 'c_1658_b_18260_d_63',
 'c_868_b_79977_d_30',
 'c_868_b_79977_d_31'}

In [9]:
graph.feed.find_stop_events(stop_id, t0)

[TimetableEvent(pattern_id=5, service_id='c_1658_b_18260_d_31', row=1, col=0, time=Timestamp('2023-04-19 14:25:00')),
 TimetableEvent(pattern_id=8, service_id='c_1658_b_18260_d_31', row=7, col=0, time=Timestamp('2023-04-19 13:20:00')),
 TimetableEvent(pattern_id=9, service_id='c_1658_b_18260_d_31', row=7, col=0, time=Timestamp('2023-04-19 13:20:00')),
 TimetableEvent(pattern_id=10, service_id='c_1658_b_18260_d_31', row=7, col=16, time=Timestamp('2023-04-19 13:50:00')),
 TimetableEvent(pattern_id=11, service_id='c_1658_b_18260_d_31', row=3, col=0, time=Timestamp('2023-04-19 13:50:00')),
 TimetableEvent(pattern_id=12, service_id='c_1658_b_18260_d_31', row=4, col=0, time=Timestamp('2023-04-19 14:50:00')),
 TimetableEvent(pattern_id=13, service_id='c_1658_b_18260_d_31', row=7, col=0, time=Timestamp('2023-04-19 13:20:00')),
 TimetableEvent(pattern_id=17, service_id='c_1658_b_18260_d_31', row=7, col=0, time=Timestamp('2023-04-19 13:20:00')),
 TimetableEvent(pattern_id=23, service_id='c_1658_

In [10]:
node = ("at_stop", stop_id, t0)
graph.adjacent_forward(node)

TypeError: find_stop_events() got an unexpected keyword argument 'find_departure'

In [39]:
for pattern_id in pattern_ids:

    j = graph.feed.stop_patterns[pattern_id].index(stop_id)

    for service_id in service_ids:
        timetable_key = (pattern_id, service_id)

        timetable = graph.feed.timetables.get(timetable_key)
        if timetable is None:
            continue

        print( time_secs )
        print( timetable.departure_times[:, j] )

        sched = timetable.departure_times[:, j]
        # search sorted
        i = np.searchsorted(sched, time_secs, side="left")
        print(i)

        break
    break

46800
[18300 20100 21900 23400 24600 25620 26580 27480 28380 29400 30540 31620
 32760 33960 35160 36360 37500 38700 39900 41100 42300 43500 44700 45900
 47100 48360 49560 50760 51960 53160 54420 55680 56580 57480 58380 59280
 60180 61080 61980 62880 63780 64620 65520 66420 67320 68340 69420 71220
 73020 74700 76500 78300 80100 81900 83700 85500]
24


In [6]:
node = ("at_stop", stop_id, t0)
graph.adjacent_forward(node)

KeyError: (99, 9158)

In [63]:
stop_pattern_ids = gtfs.stop_pattern_ids[stop_id]
stop_pattern_ids

{98, 99, 100, 106, 336, 337, 342, 343, 371, 372, 374, 510}

In [67]:
# for each stop pattern associated with this stop
for stop_pattern_id in stop_pattern_ids:
    pattern = gtfs.stop_patterns[stop_pattern_id]
    j = pattern.index(stop_id)

    # for each service_id associated with this day
    for service_id in service_ids:
        key = (stop_pattern_id, service_id)

        if key not in gtfs.schedules:
            continue

        # sched is an array of shape (n_trips, n_stops, 2)
        trip_ids, timetable = gtfs.schedules[key]

        timetable = timetable[:, j, 1] # array of departure times

        # find the first departure after time_secs
        i = np.searchsorted(timetable, time_secs, side="left")
        print(i)

        print(time_secs, timetable[i, j, 1])

        break
    break

24
46800 47100


In [7]:
# join trips and stop_times
#stop_time_trips = trips.merge(stop_times, on="trip_id")

In [9]:
# get trip_id -> stop pattern

trip_stop_patterns = stop_times.sort_values("stop_sequence").groupby(["trip_id"]) \
             .agg({"stop_id": tuple}).to_dict(orient="index")


In [10]:
# reverse to get stop pattern -> list of trip_ids

stop_pattern_trips = defaultdict(set)
for trip_id, stop_pattern in trip_stop_patterns.items():
    stop_pattern_trips[stop_pattern["stop_id"]].add(trip_id)
stop_pattern_trips = dict(stop_pattern_trips)

In [30]:
# create stop_pattern_id -> ordered list of stops

stop_patterns = list(zip(range(len(stop_pattern_trips)), stop_pattern_trips.keys()))

In [12]:
# create stop_pattern_id -> list of trip_ids

stop_pattern_id_trips = expand_pairs(zip(range(len(stop_pattern_trips)), stop_pattern_trips.values()))

In [13]:
stop_pattern_id_trips_df = pd.DataFrame(stop_pattern_id_trips, columns=["stop_pattern_id", "trip_id"])
stop_pattern_id_trips_df

,stop_pattern_id,trip_id
0,0,55295528
1,0,55295529
2,0,55295530
3,0,55295531
4,0,55295532
...,...,...
27683,546,617043053
27684,546,617043693
27685,546,617043823
27686,546,617044343


In [16]:
stop_times = stop_times.merge(stop_pattern_id_trips_df, on="trip_id").merge(trips, on="trip_id")

In [24]:
# pivot. columns: stop_id, index: trip_id, values: [arrival_time, depature_time]
tmp = stop_times[stop_times["stop_pattern_id"] == 0].sort_values("departure_time")
tmp = tmp.pivot(index="trip_id", columns="stop_id", values=["arrival_time", "departure_time"])
tmp

arrival_time                                                           
stop_id         1551   1652   1662   1672   1682   11175  27420  41904  41908   
trip_id                                                                         
55295528        21480  21600  21720  21943  22040  22980  22260  22434  22637  \
55295529        22380  22500  22620  22843  22940  23880  23160  23334  23537   
55295530        23280  23400  23520  23743  23840  24780  24060  24234  24437   
55295531        24180  24300  24420  24643  24740  25680  24960  25134  25337   
55295532        25080  25200  25320  25543  25640  26580  25860  26034  26237   
...               ...    ...    ...    ...    ...    ...    ...    ...    ...   
55296140        67080  67200  67320  67543  67640  68580  67860  68034  68237   
55296141        68280  68400  68520  68743  68840  69780  69060  69234  69437   
55296142        69480  69600  69720  69943  70040  70980  70260  70434  70637   
55296143        70680  70800  70920  71143  71240  72180  71460  71634  71837   
55296144        71880  72000  72120  72343  72440  73380  72660  72834  73037   

                departure_time                                             
stop_id   41970          1551   1652   1662   1672   1682   11175  27420   
trip_id                                                                    
55295528  22852          21480  21600  21720  21943  22040  22980  22260  \
55295529  23752          22380  22500  22620  22843  22940  23880  23160   
55295530  24652          23280  23400  23520  23743  23840  24780  24060   
55295531  25552          24180  24300  24420  24643  24740  25680  24960   
55295532  26452          25080  25200  25320  25543  25640  26580  25860   
...         ...            ...    ...    ...    ...    ...    ...    ...   
55296140  68452          67080  67200  67320  67543  67640  68580  67860   
55296141  69652          68280  68400  68520  68743  68840  69780  69060   
55296142  70852          69480  69600  69720  69943  70040  70980  70260   
55296143  72052          70680  70800  70920  71143  71240  72180  71460   
55296144  73252          71880  72000  72120  72343  72440  73380  72660   

                               
stop_id   41904  41908  41970  
trip_id                        
55295528  22434  22637  22852  
55295529  23334  23537  23752  
55295530  24234  24437  24652  
55295531  25134  25337  25552  
55295532  26034  26237  26452  
...         ...    ...    ...  
55296140  68034  68237  68452  
55296141  69234  69437  69652  
55296142  70434  70637  70852  
55296143  71634  71837  72052  
55296144  72834  73037  73252  

[177 rows x 20 columns]

In [55]:
trip_stop_pattern_id = {}
for stop_pattern_id, trips in stop_pattern_id_trips:
    for trip_id in trips:
        trip_stop_pattern_id[trip_id] = stop_pattern_id

In [56]:
trip_stop_pattern_id

{}

In [41]:
stop_service_schedule = stop_time_trips.sort_values("arrival_time").groupby(["service_id", "stop_id"]).agg({
    "arrival_time": list,
    "departure_time": list,
    "trip_id": list,
    "stop_sequence": list,
}).to_dict(orient="index")

In [42]:
stops = pd.read_csv( zf.open("stops.txt") )

In [43]:
stop_id = 11080

In [44]:

service_ids = tuple(service_dates[pd.Timestamp("2023-04-02")])
service_ids

(12202, 12034)

In [45]:
stop_service_schedule[(service_ids[0], stop_id)]

{'arrival_time': [22320,
  22620,
  24120,
  24600,
  25920,
  26520,
  27720,
  28380,
  29520,
  30360,
  31140,
  31320,
  32040,
  32940,
  33060,
  33840,
  34740,
  34860,
  35640,
  36540,
  36660,
  37440,
  38340,
  38460,
  39240,
  40200,
  40260,
  41100,
  42000,
  42060,
  42900,
  43800,
  43860,
  44700,
  45600,
  45660,
  46500,
  47400,
  47460,
  48300,
  49200,
  49260,
  50100,
  51000,
  51060,
  51900,
  52800,
  52860,
  53700,
  54600,
  54660,
  55500,
  56400,
  56460,
  57300,
  58200,
  58260,
  59100,
  60000,
  60060,
  60900,
  61800,
  61920,
  62700,
  63600,
  63720,
  64500,
  65340,
  65520,
  66420,
  67320,
  67320,
  68220,
  69120,
  69120,
  70020,
  70920,
  70920,
  71820,
  72720,
  72720,
  73620,
  74520,
  74520,
  75300,
  76200,
  76320,
  77100,
  77940,
  78120,
  78840,
  79920,
  80580,
  81720,
  82320,
  83520,
  84120,
  85320,
  85920,
  87000,
  87720,
  88560,
  90120,
  90600,
  91920,
  94620,
  98280,
  102240,
  103380],
